In [ ]:
import pybullet as p
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
from rbdyn.parsers import *

DT = 0.001 #s
MOVEMENT_TIME = 10000 #ms
URDF = "./resource/kuka_iiwa.urdf"

def get_rot(T):
    R = e.Matrix3d()
    for row in range(R.rows()):
        for col in range(R.cols()):
            R.coeff(row, col, T.coeff(row, col))
    return R

def get_pos(T):
    pos = e.Vector3d()
    for i in range(pos.rows()):
        pos[i] = T.coeff(i,3)
    return pos

def get_col(R, c=0):
    assert c <= R.cols(), "column index out of bounds"
    assert R.rows() == R.cols(), "square matrix expected"
    assert R.rows() == 3, "3x3 matrix expected"
    v = e.Vector3d()
    for i in range(R.rows()):
        v[i] = R.coeff(i,c)
    return v

def get_orientation_error(R, R_d):
    '''Calculates orientation error
    @param R Current orientation in Mat3d
    @param R_d Desired orientation in Mat3d
    @return del_phi "R_d-R" error
    '''
    c1 = get_col(R,0)
    c2 = get_col(R,1)
    c3 = get_col(R,2)
    d1 = get_col(R_d,0)
    d2 = get_col(R_d,1)
    d3 = get_col(R_d,2)
    
    return 1.0/2.0*(c1.cross(d1)+c2.cross(d2)+c3.cross(d3))
    
    
def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    pos = s.translation();
    
    for row in range(R.rows()):
        for col in range(R.cols()):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(pos.rows()):
        m4d.coeff(row, 3, pos[row]);
        
    return m4d

def inv_spd_sr(mat, k=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param tolerance Tolerance to the minimum singular value that can go through reciprocal operation
    @return Homogeneous transform matrix
    '''
    mat_np = np.array(mat)
    reg = k * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg)))
    
# import robot as global variable
r = from_urdf_file(URDF)
print("Imported " + r.name.decode("utf-8"))

# set gravity direction (this is the acceleration at base joint for RNEA)
r.mbc.gravity = e.Vector3d(0,0,9.81)
r.mbc.zero(r.mb)

# setup physics
physicsClient = p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
p.setGravity(0,0,-9.81)
p.setRealTimeSimulation(0)
p.setTimeStep(DT)

# import robot
plane_id = p.loadURDF("plane.urdf")
robot_id = p.loadURDF(URDF, [0,0,0], p.getQuaternionFromEuler([0,0,0]), 
                     flags=(p.URDF_USE_INERTIA_FROM_FILE|p.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS))

# parse number of joints except fixed joint
print("Auto parse revolute and fixed joints only")
joints_id = []
for i in range(p.getNumJoints(robot_id)):
    if(p.getJointInfo(robot_id, i)[2] == p.JOINT_REVOLUTE):
        joints_id.append(i)
joints_id = np.array(joints_id)
n_dof = len(joints_id)

# gains
Kp = e.MatrixXd(np.diag([10000,10000,10000,40000,40000,40000]))
Kd = e.MatrixXd(np.diag([200,200,200,400,400,400]))

# selection matrix to swap linear and angular components
S = e.MatrixXd(6,6)
S.coeff(0,3,1)
S.coeff(1,4,1)
S.coeff(2,5,1) # linear
S.coeff(3,0,1)
S.coeff(4,1,1)
S.coeff(5,2,1) # angular

# task definitions
body_name_1 = "lbr_iiwa_link_7"
S1 = S

# setup configuration
p.setJointMotorControlArray(robot_id, joints_id, p.VELOCITY_CONTROL, forces=[0] * n_dof)
q_home = np.zeros(n_dof)
for i in joints_id:
    p.resetJointState(robot_id, i, targetValue=q_home[i])
    p.enableJointForceTorqueSensor(robot_id, i)
q_home = e.VectorXd(q_home)

# shows the link frame defined by <inertial><origin> in urdf, -1 is the first link
com_ee = list(p.getDynamicsInfo(robot_id, n_dof-1)[3]) # last link
start = [-x for x in com_ee]
end_x = start.copy()
end_y = start.copy()
end_z = start.copy()
end_x[0] += 0.3
end_y[1] += 0.3
end_z[2] += 0.3
# shows the frame at the <inertial><origin> rather than the joint, using com_ee, we can adjust to the joint
p.addUserDebugLine(start, end_x, lineColorRGB=[1, 0, 0], lineWidth=2, parentObjectUniqueId=robot_id, parentLinkIndex=n_dof-1)
p.addUserDebugLine(start, end_y, lineColorRGB=[0, 1, 0], lineWidth=2, parentObjectUniqueId=robot_id, parentLinkIndex=n_dof-1)
p.addUserDebugLine(start, end_z, lineColorRGB=[0, 0, 1], lineWidth=2, parentObjectUniqueId=robot_id, parentLinkIndex=n_dof-1)

# setup camera
p.resetDebugVisualizerCamera(5,0,0,[0,0,1.18])

def set_q_in_mbc(q):
    for i in range(0, n_dof):
        r.mbc.q[i+1][0] = q[i]

def set_dq_in_mbc(dq):
    for i in range(0, n_dof):
        r.mbc.alpha[i+1][0] = dq[i]
    
# gains
Kp_joint = e.MatrixXd(np.diag([8000]*n_dof))
Kd_joint = e.MatrixXd(np.diag([180]*n_dof))
Ko_joint = e.MatrixXd(np.diag([100]*n_dof))
    
# observer
residual = e.VectorXd([0] * n_dof)
residual_int = e.VectorXd([0] * n_dof)
    
# simulate torque control
for i in range(10000):
    joint_states = p.getJointStates(robot_id, joints_id)
    
    # update q and dq states from simulation
    q_bullet = [joint_states[i][0] for i in joints_id]
    dq_bullet = [joint_states[i][1] for i in joints_id]

    # update states into mbc
    set_q_in_mbc(q_bullet)
    set_dq_in_mbc(dq_bullet)
    
    # updates states into "eigen" representation
    q = e.VectorXd(q_bullet)
    dq = e.VectorXd(dq_bullet)

    ############################### forward kinematics to update model ###############################
    rbd.forwardKinematics(r.mb, r.mbc)
    rbd.forwardVelocity(r.mb, r.mbc)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(0,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * r.mbc.bodyPosW[body_id_from_name(body_name_1, r.mb.bodies())])
    
    # jacobian, dJacobian [angular; linear]
    jac = rbd.Jacobian(r.mb, body_name_1.encode('utf-8'), ofsPos)
    J1_temp = e.MatrixXd(6, r.mb.nrDof());
    dJ1_temp = e.MatrixXd(6, r.mb.nrDof());
    jac.fullJacobian(r.mb, jac.jacobian(r.mb, r.mbc), J1_temp);
    jac.fullJacobian(r.mb, jac.jacobianDot(r.mb, r.mbc), dJ1_temp);
    J1 = S*J1_temp
    dJ1 = S*dJ1_temp
    
    ############################### forward dynamics to update model ###############################
    fd = rbd.ForwardDynamics(r.mb)
    
    # mass matrix & nonlinear effects 
    fd.computeH(r.mb, r.mbc)
    M = fd.H()
    fd.computeC(r.mb, r.mbc)
    h = fd.C()
    
    # coriolis matrix
    cor = rbd.Coriolis(r.mb)
    C = cor.coriolis(r.mb, r.mbc)
    
    # gravity
    set_dq_in_mbc([0] * n_dof)
    rbd.forwardKinematics(r.mb, r.mbc)
    fd = rbd.ForwardDynamics(r.mb)
    fd.computeC(r.mb, r.mbc)
    g = fd.C()
    set_dq_in_mbc(dq_bullet)
    rbd.forwardKinematics(r.mb, r.mbc)
    rbd.forwardVelocity(r.mb, r.mbc)
    
    # apply force
    p.applyExternalForce(robot_id, joints_id[-1], (0,0,1), (0,0,0), p.LINK_FRAME)

    # control
    err = e.VectorXd([0,np.pi/4,0,-np.pi/2,0,np.pi/4,0])-q
    derr = -dq
    tau = e.VectorXd(M*(Kp_joint*err+Kd_joint*derr)) + h

    # momentum observer
    beta = g - C.transpose()*dq
    residual_int += (residual+tau+C.transpose()*dq-g)*DT
    residual = Ko_joint*(M*dq-residual_int)
    Jc = e.MatrixXd(J1)
    JcT = Jc.transpose()
    JcT_inv = e.MatrixXd(np.linalg.pinv(JcT))
    f = JcT_inv*residual
    
    # printing
    if i % 200 == 0:
        print("r = {}".format(residual.transpose()))
        print("F = {}".format(f.transpose()))
    
    # command system
    for i in joints_id:
        p.setJointMotorControl2(robot_id, i, p.TORQUE_CONTROL, force=tau[i])
        
    # step simulation
    p.stepSimulation()
    time.sleep(DT)
    
time.sleep(1)
p.disconnect()

pybullet build time: Mar 26 2022 03:00:52


Imported lbr_iiwa
Auto parse revolute and fixed joints only
r =    -36.8925    -3558.62    -43.1204     1377.46    -26.2169    -21.8354 6.15324e-12
F =      -6432.8  4.89162e+15 -2.13836e+15  3.07634e+15      1250.92     -659.345
r =  2.04391e-05     0.573235 -6.76477e-05    -0.278905 -0.000204979   -0.0273717 -0.000944753
F =   -0.045946  -0.0015854    -1.03698 0.000254785  -0.0273717  0.00081028
r = -2.46201e-05     0.579815 -1.64828e-05    -0.282818  5.66849e-06   -3.348e-05 -8.90853e-05
F = -4.28579e-05 -0.000194708     -1.00006  4.67939e-05 -3.34743e-05   7.2044e-05
r = -2.50989e-05     0.579815 -1.64737e-05    -0.282818  6.53591e-07 -2.29839e-08 -9.05706e-06
F =  1.59114e-05 -5.87644e-05           -1  4.81048e-06 -2.24004e-08  7.30537e-06


In [ ]:
import pybullet as b
import time
import pybullet_data
import eigen as e
import rbdyn as rbd
import sva as s
import numpy as np
from rbdyn.parsers import *

DT = 0.001 #s
MOVEMENT_TIME = 5000 #ms
URDF = "./resource/four_dof.urdf"

def body_id_from_name(name, bodies):
    '''Gets the body Id from the body name
    @param name The name of the body
    @param bodies The set of bodies provided by the multibody data structure
    @return Id of the body, -1 if not found
    '''
    for bi, b in enumerate(bodies):
        if (b.name().decode("utf-8") == name):
            return bi
    return -1

def sva_to_affine(s):
    '''Converts a spatial transform matrix to a homogeneous transform matrix
    @param s Spatial transform
    @return Homogeneous transform matrix
    '''
    m4d = e.Matrix4d.Identity()
    R = s.rotation().transpose()
    p = s.translation()
    
    for row in range(3):
        for col in range(3):
            m4d.coeff(row, col, R.coeff(row, col))
    for row in range(3):
        m4d.coeff(row, 3, p[row])
        
    return m4d

def inv_spd_edls(mat, sigma_neg, sigma_pos):
    """
    Exponentially damped least squares for inversion from https://doi.org/10.1177/0278364917698748
    """
    pass

def inv_spd_sr(mat, reg=1e-4):
    '''Converts an inverse into an SVD problem with singularity robust inverse
    @param mat Matrix to be inversed
    @param reg Regularization value for the singularity robust inverse
    @return inverse of matrix
    '''
    mat_np = np.array(mat)
    reg_mat = reg * np.identity(np.size(mat_np,0))
    
    return e.MatrixXd(mat_np.transpose().dot(np.linalg.inv(mat_np.dot(mat_np.transpose()) + reg_mat)))

# import robot as global variable
p = from_urdf_file(URDF)
print("Imported " + p.name.decode("utf-8"))
# set gravity direction (this is the acceleration at base joint for RNEA)
p.mbc.gravity = e.Vector3d(0,0,9.81)
p.mbc.zero(p.mb)

# setup physics
physicsClient = b.connect(b.GUI)
b.setAdditionalSearchPath(pybullet_data.getDataPath()) #used by loadURDF
b.setGravity(0,0,-9.81)
b.setRealTimeSimulation(0)
b.setTimeStep(DT)

# import robot
planeId = b.loadURDF("plane.urdf")
startPos = [0,0,1]
startOrientation = b.getQuaternionFromEuler([0,0,0])
loadFlag = b.URDF_USE_INERTIA_FROM_FILE | b.URDF_USE_SELF_COLLISION_EXCLUDE_ALL_PARENTS
robotId = b.loadURDF(URDF, startPos, startOrientation, flags=loadFlag)
nDof = b.getNumJoints(robotId)
jointsId = range(nDof)

# setup configuration
b.setJointMotorControlArray(robotId, jointsId, b.VELOCITY_CONTROL, forces=[0] * nDof)
q_home = [0.6524786685190415, -0.7891557752882681, 0.8792139557391727, 1.3774730897711231]
for i in jointsId:
    b.resetJointState(robotId, i, targetValue=q_home[i])
    
# simulate torque control
for i in range(100000):
    joint_states = b.getJointStates(robotId, jointsId)
    
    # read state feedback
    q = [joint_states[i][0] for i in jointsId]
    dq = [joint_states[i][1] for i in jointsId]
    
    # update state in model
    p.mbc.q = [[], [q[0]], [q[1]], [q[2]], [q[3]]]
    p.mbc.alpha = [[], [dq[0]], [dq[1]], [dq[2]], [dq[3]]]

    # forward kinematics
    rbd.forwardKinematics(p.mb, p.mbc)
    rbd.forwardVelocity(p.mb, p.mbc)
    
    # convert
    q = e.VectorXd(q)
    dq = e.VectorXd(dq)
    
    # pose
    ofsRot = e.Matrix3d.Identity()
    ofsPos = e.Vector3d(1,0,0)
    T1 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name("link_4", p.mb.bodies())])
    T2 = sva_to_affine(s.PTransformd(ofsRot.transpose(), ofsPos) * p.mbc.bodyPosW[body_id_from_name("link_2", p.mb.bodies())])
    
    # select xz coordinates
    S = e.MatrixXd.Zero(2,6)
    S.coeff(0,3,1)
    S.coeff(1,5,1)
    
    # jacobian, dJacobian [angular; linear]
    bodyName = "link_4"
    jac = rbd.Jacobian(p.mb, bodyName.encode('utf-8'), ofsPos)
    tempJ1 = e.MatrixXd(6, p.mb.nrDof())
    tempdJ1 = e.MatrixXd(6, p.mb.nrDof())
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ1);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ1);
    J1 = S*tempJ1
    dJ1 = S*tempdJ1
    
    # jacobian, dJacobian [angular; linear]
    bodyName = "link_2"
    jac = rbd.Jacobian(p.mb, bodyName.encode('utf-8'), ofsPos)
    tempJ2 = e.MatrixXd(6, p.mb.nrDof())
    tempdJ2 = e.MatrixXd(6, p.mb.nrDof())
    jac.fullJacobian(p.mb, jac.jacobian(p.mb, p.mbc), tempJ2);
    jac.fullJacobian(p.mb, jac.jacobianDot(p.mb, p.mbc), tempdJ2);
    J2 = S*tempJ2
    dJ2 = S*tempdJ2
    
    # mass matrix
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeH(p.mb, p.mbc);
    M = fd.H()

    # nonlinear effects
    fd = rbd.ForwardDynamics(p.mb)
    fd.computeC(p.mb, p.mbc);
    h = fd.C()
    
    # compute Minv
    Minv = M.inverse()
    
    # compute lambda1, Jbar1, Nprec1
    lambda1 = inv_spd_sr(J1*Minv*J1.transpose(),1e-6)
    J1bar = Minv*J1.transpose()*lambda1
    Nprec1 = e.MatrixXd.Identity(nDof, nDof) - J1bar*J1
    
    # compute lambda21, Jbar21
    J21 = J2*Nprec1
    lambda21 = inv_spd_sr(J21*Minv*J21.transpose(),1e-6)
    J21bar = Minv*J21.transpose()*lambda21
    Nprec2 = Nprec1 * (e.MatrixXd.Identity(nDof, nDof) - J21bar*J21)
    
    # change desired position for secondary task
    if i < MOVEMENT_TIME:
        xdes = 1 + (i*(10.0-1)/MOVEMENT_TIME)
        zdes = 1 + (i*(10.0-1)/MOVEMENT_TIME)
    
    # error
    err1 = e.VectorXd(2)
    err1[0] = 2 - T1.coeff(0,3)
    err1[1] = 2 - T1.coeff(2,3)
    derr1 = -J1*dq
    err2 = e.VectorXd(2)
    err2[0] = xdes - T2.coeff(0,3)
    err2[1] = zdes - T2.coeff(2,3)
    derr2 = -J2*dq

    # control of task 1
    tau_1 = e.VectorXd(J1.transpose()*lambda1*(400*err1+40*derr1-dJ1*dq))
    # control of task 2 (0 nullspace damping)
#     tau_2 = e.VectorXd(4)
    # control of task 2 (to singularity)
    tau_2 = e.VectorXd(J21.transpose()*lambda21*(400*err2+40*derr2-dJ2*dq-J2*Minv*tau_1))
    tau = tau_1 + Nprec1.transpose()*tau_2 + h

    # command system:
    for i in jointsId:
        b.setJointMotorControl2(robotId, i, b.TORQUE_CONTROL, force=tau[i])
    
    b.stepSimulation()
    time.sleep(DT)
b.disconnect()

pybullet build time: Mar 26 2022 03:00:52


Imported FourJointRobot
